In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import dgl.data
dataset = dgl.data.CoraGraphDataset()
print("Number of categories :" , dataset.num_classes)


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories : 7


In [ ]:
g = dataset[0]  #The Cora dataset used here consists only of one single graph.

In [ ]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


This tutorial will build a two-layer Graph Convolutional Network (GCN). Each layer computes new node representations by aggregating neighbour information.

In [ ]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

Training the GCN

In [ ]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In epoch 0, loss: 1.946, val acc: 0.110 (best 0.110), test acc: 0.119 (best 0.119)
In epoch 5, loss: 1.894, val acc: 0.552 (best 0.576), test acc: 0.577 (best 0.577)
In epoch 10, loss: 1.816, val acc: 0.484 (best 0.576), test acc: 0.494 (best 0.577)
In epoch 15, loss: 1.717, val acc: 0.552 (best 0.576), test acc: 0.566 (best 0.577)
In epoch 20, loss: 1.595, val acc: 0.628 (best 0.628), test acc: 0.653 (best 0.653)
In epoch 25, loss: 1.454, val acc: 0.672 (best 0.672), test acc: 0.699 (best 0.699)
In epoch 30, loss: 1.299, val acc: 0.678 (best 0.680), test acc: 0.713 (best 0.713)
In epoch 35, loss: 1.137, val acc: 0.692 (best 0.696), test acc: 0.728 (best 0.721)
In epoch 40, loss: 0.974, val acc: 0.722 (best 0.722), test acc: 0.738 (best 0.738)
In epoch 45, loss: 0.818, val acc: 0.732 (best 0.732), test acc: 0.746 (best 0.746)
In epoch 50, loss: 0.676, val acc: 0.738 (best 0.738), test acc: 0.746 (best 0.746)
In epoch 55, loss: 0.551, val acc: 0.742 (best 0.742), test acc: 0.755 (best 0